In [2]:
#Basic imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import time
import os
from collections import Counter
import json
from pathlib import Path
from datetime import datetime
import pickle

#Tensorflow impocarts
import tensorflow as tf
from tensorflow import keras
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.callbacks import CSVLogger

In [ ]:
def build_model(hyper_params,vocab, cuDNN=False):
    
    vocab_size=vocab['vocab_size_route']
    embed_size = hyper_params['embed_size_route'] #embedding size 
    rnn_units = hyper_params['rnn_units'] #number of neurons per layer
    dropout_val = hyper_params['dropout_val']
    rec_dropout_val = hyper_params['rec_dropout_val']

    if cuDNN:
        #Build model
        model = keras.models.Sequential([
            keras.layers.Embedding(input_dim=vocab_size, output_dim=embed_size, mask_zero=False, #mask_zero tells the following layers
                                  input_shape = [None]),                                        #and the loss function to ignore the 
            tf.compat.v1.keras.layers.CuDNNLSTM(units=rnn_units, return_sequences=True),
            tf.compat.v1.keras.layers.CuDNNLSTM(units=rnn_units, return_sequences=True),
            keras.layers.TimeDistributed(keras.layers.Dense(vocab_size,
                                                           activation='softmax'))
        ])
        
        #Compile
        model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=["accuracy"])

    else:
        #Build model
        model = keras.models.Sequential([
            keras.layers.Embedding(input_dim=vocab_size, output_dim=embed_size, mask_zero=False, #mask_zero tells the following layers
                                  input_shape = [None]),                                        #and the loss function to ignore the 
            keras.layers.LSTM(rnn_units, return_sequences=True,                                 #padding
                             dropout = dropout_val, recurrent_dropout=rec_dropout_val),
            keras.layers.LSTM(rnn_units, return_sequences=True,
                             dropout = dropout_val, recurrent_dropout=rec_dropout_val),
            keras.layers.TimeDistributed(keras.layers.Dense(vocab_size,
                                                           activation='softmax'))
        ])
        
        
        
        #Compile
        model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=["accuracy"])
    
    return model

In [ ]:
def build_model_all(hyper_params, vocab):

    #Hyperparameters
    embed_size_route = hyper_params['embed_size_route'] #embedding size 
    embed_size_ndays = hyper_params['embed_size_ndays'] #embedding size 
    rnn_units = hyper_params['rnn_units'] #number of neurons per layer
    dropout_val = hyper_params['dropout_val']
    rec_dropout_val = hyper_params['rec_dropout_val']
    n_hidden_layers = hyper_params['n_hidden_layers'] 
    max_sen_len = hyper_params['max_sen_len']

    #Define input layers
    inputs = {}
    inputs['route_id'] = keras.Input(shape=[None])  #shape=[max_sen_len-1]
    inputs['ndays'] = keras.Input(shape=[None])
    inputs['rating_yds'] = keras.Input(shape=[None])
    inputs['rating_vscale'] = keras.Input(shape=[None])
    inputs['rating_misc'] = keras.Input(shape=[None])
    inputs['routetype'] = keras.Input(shape=[None])
    

    #Define embedding layers
    emd_route = keras.layers.Embedding(input_dim=vocab['vocab_size_route'],
                                       output_dim=embed_size_route)(inputs['route_id'])
    
    emd_ndays = keras.layers.Embedding(input_dim=vocab['vocab_size_bins'],
                                       output_dim=embed_size_ndays)(inputs['ndays'])
    
    emd_yds = keras.layers.Embedding(input_dim=vocab['vocab_size_yds'],
                                       output_dim=embed_size_ndays)(inputs['rating_yds'])
    
    emd_v = keras.layers.Embedding(input_dim=vocab['vocab_size_vscale'],
                                       output_dim=embed_size_ndays)(inputs['rating_vscale'])
    
    emd_misc = keras.layers.Embedding(input_dim=vocab['vocab_size_misc'],
                                       output_dim=embed_size_route)(inputs['rating_misc'])
    
    emd_type = keras.layers.Embedding(input_dim=vocab['vocab_size_type'],
                                       output_dim=embed_size_ndays)(inputs['routetype'])
    
    
    
  
    #Concatenate embedding layers
    emd_concat = tf.keras.layers.Concatenate()([emd_route,
                                                emd_ndays,
                                                emd_yds,
                                                emd_v,
                                                emd_type])

    #Batch normalization to speed up training
    emd_concat = keras.layers.BatchNormalization()(emd_concat)

    #LSTM layers (start with 1 layer, optionally 2)

    RNN = tf.compat.v1.keras.layers.CuDNNLSTM(units=rnn_units,
                                              return_sequences=True)(emd_concat)
    if n_hidden_layers == 2:
        RNN = tf.compat.v1.keras.layers.CuDNNLSTM(units=rnn_units,
                                                  return_sequences=True)(RNN)

    #Batch norm again
    RNN = keras.layers.BatchNormalization()(RNN)

    #Last layer - probabilities
    output = keras.layers.Dense(vocab_size, activation='softmax')(RNN) #make sure this is actually working ....                                            

    #Model
    model = tf.keras.Model(inputs, output)
    model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=["accuracy"])


    return model

In [3]:
#Load tokenizer, word_index, vocab_size:
fp_tokenizer = 'tokenizer_cuDNN_230203-023518.pickle'
fp_wordindex = 'wordindex_cuDNN_230203-023518.json'
with open(fp_tokenizer, "rb") as fp:
    tokenizer = pickle.load(fp)
with open(fp_wordindex, "rb") as fp:
    word_index = json.load(fp)
vocab_size = len(word_index) + 1

#Load datasets
train_set_route = tf.data.Dataset.load('Data/train_set_route')
val_set_route = tf.data.Dataset.load('Data/val_set_route')
train_set_ndays= tf.data.Dataset.load('Data/train_set_ndays')
val_set_ndays = tf.data.Dataset.load('Data/val_set_ndays')
train_set_all = tf.data.Dataset.load('Data/train_set_all')
val_set_all = tf.data.Dataset.load('Data/val_set_all')



### Training route only model

In [ ]:
#Build model
route_rec_rnn = build_model(hyper_params, vocab)

s = 'route'

#Define paths and if you're restarting training or not
training_restart = False

if training_restart:
    #Load weights If training was interupted:
    date = '' #input date!
    checkpoint_path = "training_routerec_rnn_{s}_{date}/cp.ckpt".format(s=s, date=date)
    checkpoint_dir = os.path.dirname(checkpoint_path)
    model.load_weights(checkpoint_path)
    
else:
    #Define path for callback info. and define callback
    date = datetime.now().strftime("%y%m%d-%H%M%S")
    checkpoint_path = "training_routerec_rnn_{}/cp.ckpt".format(date)
    checkpoint_dir = os.path.dirname(checkpoint_path)
    
#Callbacks
csv_logger = CSVLogger("model_history_log_{}.csv".format(date), append=True) #saves for every epoch
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1,
                                                 period=20) #only saves every 20 epochs! Saves time. 

#Train model
route_rec_rnn.fit(train_set_route,
          epochs = hyper_params['epochs'],
          validation_data = val_set_route,
          callbacks = [csv_logger, cp_callback])

#Save entire model in checkpoint folder when finished training 
# Saving the entire model
model_json = model.to_json()
with open('model_{s}_{date}.json'.format(date=date, s=s), "w") as json_file:
    json_file.write(model_json)
    
model.save_weights("model_weights_{s}_{date}.h5".format(s=s, date=date))

### Training route/ndays  model

In [ ]:
#Build model
route_rec_rnn = build_model(hyper_params, vocab)

s = 'ndays'

#Define paths and if you're restarting training or not
training_restart = False

if training_restart:
    #Load weights If training was interupted:
    date = '' #input date!
    checkpoint_path = "training_routerec_rnn_{s}_{date}/cp.ckpt".format(s=s, date=date)
    checkpoint_dir = os.path.dirname(checkpoint_path)
    model.load_weights(checkpoint_path)
    
else:
    #Define path for callback info. and define callback
    date = datetime.now().strftime("%y%m%d-%H%M%S")
    checkpoint_path = "training_routerec_rnn_{}/cp.ckpt".format(date)
    checkpoint_dir = os.path.dirname(checkpoint_path)
    
#Callbacks
csv_logger = CSVLogger("model_history_log_{}.csv".format(date), append=True) #saves for every epoch
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1,
                                                 period=20) #only saves every 20 epochs! Saves time. 

#Train model
route_rec_rnn.fit(train_set_route,
          epochs = hyper_params['epochs'],
          validation_data = val_set_route,
          callbacks = [csv_logger, cp_callback])


#Save entire model in checkpoint folder when finished training 
# Saving the entire model
model_json = model.to_json()
with open('model_{s}_{date}.json'.format(date=date, s=s), "w") as json_file:
    json_file.write(model_json)
    
model.save_weights("model_weights_{s}_{date}.h5".format(s=s, date=date))

### Training all features model

In [ ]:
#Build model
route_rec_rnn = build_model(hyper_params, vocab)

s = 'all'

#Define paths and if you're restarting training or not
training_restart = False
if training_restart:
    #Load weights If training was interupted:
    date = '' #input date!
    checkpoint_path = "training_routerec_rnn_{s}_{date}/cp.ckpt".format(s=s, date=date)
    checkpoint_dir = os.path.dirname(checkpoint_path)
    model.load_weights(checkpoint_path)
    
else:
    #Define path for callback info. and define callback
    date = datetime.now().strftime("%y%m%d-%H%M%S")
    checkpoint_path = "training_routerec_rnn_{}/cp.ckpt".format(date)
    checkpoint_dir = os.path.dirname(checkpoint_path)
    
#Callbacks
csv_logger = CSVLogger("model_history_log_{}.csv".format(date), append=True) #saves for every epoch
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1,
                                                 period=20) #only saves every 20 epochs! Saves time. 

#Train model
route_rec_rnn.fit(train_set_route,
          epochs = hyper_params['epochs'],
          validation_data = val_set_route,
          callbacks = [csv_logger, cp_callback])

    
#Save entire model in checkpoint folder when finished training 
# Saving the entire model
model_json = model.to_json()
with open('model_{s}_{date}.json'.format(date=date, s=s), "w") as json_file:
    json_file.write(model_json)
    
model.save_weights("model_weights_{s}_{date}.h5".format(s=s, date=date))